In [ ]:
import numpy as np
import pandas as pd
import time
import random
from helper_functions import *

In [ ]:
#problem_instance = "inst_tuning\heur040_n_300_m_13358"
instance_type = "test_instances/" #"inst_competition/" #"inst_tuning/" # 
problem_instance = "heur002_n_100_m_3274" #"heur040_n_300_m_13358" #"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"

In [ ]:
nodes, edges, s = create_problem_instance(path)
is_splex(nodes = nodes, plex_number = 2, s=s)

# Construction heuristic

In [ ]:
constr_nodes, constr_edges, constr_score = construction_heuristic(nodes,edges,s)
print("number of plexes: "+ str(len(constr_nodes["splex"].unique())))
print("score: " +str(constr_score))
print(constr_edges)

# Randomized Construction heuristic

In [ ]:
rand_nodes, rand_edges, rand_score = randomized_greedy(nodes, edges, s, alpha = 0.75)
print("number of plexes: "+ str(len(rand_nodes["splex"].unique())))
print("score: " +str(rand_score))

# extract 1 node neighborhood

In [ ]:
extract_nodes, extract_edges, extract_score = extract1node(constr_nodes, constr_edges, constr_score, s, step = "best")
print("number of plexes: "+ str(len(extract_nodes["splex"].unique())))
print("score: " +str(extract_score))

# merge 2 plexes neighborhood

In [ ]:
merged_nodes, merged_edges, merged_score = merge2plexes(constr_nodes, constr_edges, constr_score, s, 
                                                        step = "best", add_neg_edges = True)
print("number of plexes: "+ str(len(merged_nodes["splex"].unique())))
print("score: " +str(merged_score))


In [ ]:
merged_nodes2, merged_edges2, merged_score2 = merge2plexes(constr_nodes, constr_edges, constr_score, s, 
                                                        step = "best", add_neg_edges = False)
print("number of plexes: "+ str(len(merged_nodes2["splex"].unique())))
print("score: " +str(merged_score2))

## do it again on the new best solution

In [ ]:
merged_nodes3, merged_edges3, merged_score3 = merge2plexes(merged_nodes, merged_edges, merged_score, s, step = "best")
print("number of plexes: "+ str(len(merged_nodes2["splex"].unique())))
print("score: " +str(merged_score2))

# einmal alles (mit scharf)

In [ ]:
instance_type = "inst_competition/"#"test_instances/" #"inst_competition/" #"inst_tuning/" # 
problem_instance = "heur051_n_300_m_20122" #"heur040_n_300_m_13358" #"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"
nodes, edges, s = create_problem_instance(path)
print(edges)
constr_nodes, constr_edges, constr_score = construction_heuristic(nodes, edges, s)
write_solution(constr_edges, problem_instance, algorithm = "construction")
print(constr_edges)

In [ ]:
def vnd(solution: Solution, neighborhoods: list[Neighborhood]) -> Solution:
    current_best: Solution = solution
    
    i = 0 # First neighborhood
    while i < len(neighborhoods):
        current_neighborhood: Neighborhood = neighborhoods[i]
        local_optimum = current_neighborhood.get_improvement(current_best, ImprovementType.BEST)

        if local_optimum.get_weight() < current_best.get_weight():
            current_best = local_optimum
            i = 0
        else:
            i += 1

    return current_best

In [ ]:
problem = Problem(path)
plex_merge_neighborhood = Neighborhood(lambda sol, step: merge2plexes(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step))
add_edge_neighborhood = Neighborhood(lambda sol, step: add_edge_within_plex(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step))
remove_edge_neighborhood = Neighborhood(lambda sol, step: remove_edge(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step))
extract_node_neighborhood = Neighborhood(lambda sol, step: extract1node(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step))

inital_solution = problem.get_heuristic_solution()
local_best = plex_merge_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)
print(inital_solution.get_weight())
print(local_best.get_weight())

In [ ]:
problem = Problem(path)
solution = grasp(problem, plex_merge_neighborhood, 20)
print(solution.get_weight())

In [ ]:
problem = Problem(path)
inital_solution = problem.get_heuristic_solution()
local_optimum = vnd(inital_solution, [plex_merge_neighborhood, add_edge_neighborhood, remove_edge_neighborhood, extract_node_neighborhood])
print(inital_solution.get_weight())
print(local_optimum.get_weight())

In [ ]:
plex_merge_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)
add_edge_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)
remove_edge_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)
extract_node_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)